<a href="https://colab.research.google.com/github/LucasHenrique-dev/projeto-QTS/blob/main/desenvolvimento/GeradrorQTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importar Bibliotecas

In [1]:
import time
import random
import math
import sys
# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Importar Informações

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_id = '1jJflIN2sOFjynx_qS8jp27uDncMGrcpj' #<-- You add in here the id from you google drive file, you can find it


download = drive.CreateFile({'id': file_id})


# Download the file to a local disc
download.GetContentFile('info_curso.txt')

##Explorar Dados do Curso

In [3]:
curso = {}
arquivo = open('/content/info_curso.txt')

inicio_aulas, fim_aulas = arquivo.readline().strip().split(',')
curso.setdefault("periodo_aula", (inicio_aulas, fim_aulas))
for linha in arquivo:
  disciplina, hora_aula, hora_aula_semanal = linha.split(',')
  curso.setdefault(disciplina, (int(hora_aula), int(hora_aula_semanal)))

In [4]:
curso

{'periodo_aula': ('7:00', '12:00'),
 'SO': (1, 4),
 'MF': (1, 4),
 'OC': (1, 4),
 'CO': (1, 4),
 'IA': (1, 4)}

## Funções Auxiliares

In [5]:
def converter_minutos(hora):
  t = time.strptime(hora, '%H:%M')
  minutos = t[3] * 60 + t[4]
  return minutos

In [6]:
# NECESSÁRIO APROMORAR A FUNÇÃO PARA ACEITAR AULAS COMEÇANDO ÀS 5 DA MANHÃ E INDO ATÉ ÀS 4 DA MANHÃ
def calcular_disciplinas_dia(curso, segmentacao_horario):
  inicio, fim = curso['periodo_aula']
  inicio, fim = converter_minutos(inicio), converter_minutos(fim)
  return ((fim - inicio)/segmentacao_horario) + 1

In [25]:
def calcular_horas_vagas(disciplinas):
  return (''.join(disciplinas).replace("NA", ' ')).strip().count(" ")

In [7]:
dias_semana = ['Segunda','Terça','Quarta','Quinta','Sexta']

## Objetivo

O algoritmo visará montar um quadro de trabalho semanal contendo o menor **Tempo de Espera Entre Aulas** possível

Tempo de Espera Entre Aulas: horário de aula vaga entre aulas

##Definir Cromossomo

Definir a sequência genética de formação de um cromossomo

Os genes determinam aspectos de um indivíduo e o cromossomo é o conjunto dessas características o qual forma o indivíduo (possível solução do problema)

A ordem será os dias da semana contendo as disciplinas lecionadas no dia

[Segunda:(..disciplinas), Terça:(..disciplinas), Quarta:(..disciplinas), Quinta:(..disciplinas), Sexta:(..disciplinas)]

Ex.: [(SO,SO,SO,SO,MF,MF),(MF,MF,OC,OC,OC,OC),(CO,CO,CO,CO,IA,IA),(IA,IA,NA,NA,NA,NA),(NA,NA,NA,NA,NA,NA)]

In [8]:
disciplinas = {0:('NA', 'Horário Vago'),
               1:('SO', 'Sistemas Operacionais'),
               2:('MF', 'Métodos Formais'),
               3:('OC', 'Organização de Computadores'),
               4:('CO', 'Compiladores'),
               5:('IA', 'Inteligência Artificial')}

segmentacao_horario = 60                                                          # Divisão do horário no QTS (minutos)
disciplinas_dia = int(calcular_disciplinas_dia(curso, segmentacao_horario))       # Retorna quantidade de sessões destinadas às disciplinas em um dia

##Impressão de Quadro de Trabalho Semanal

In [30]:
def imprimir_qts(calendario):
  dia_id = 0
  espera_total = 0
  for i in range((len(calendario) ) // disciplinas_dia):
      dia = dias_semana[i]
      comeco_semana = i*disciplinas_dia
      
      disciplinas_semana = calendario[comeco_semana:comeco_semana+disciplinas_dia]                        # coleta as disciplinas da semana
      ordem_disciplinas = [disciplinas[disciplina][0] for disciplina in disciplinas_semana]               # traduz números em disciplinas

      dia_id += 1                                                                                         # próximo dia da semana
      print(f'|{dia:7}| {", ".join(ordem_disciplinas)}|')
      
      espera_total += calcular_horas_vagas(ordem_disciplinas)

  print('\nTempo Total de Espera Entre Aulas: ', espera_total)

In [31]:
imprimir_qts([1,1,1,1,2,2, 2,2,3,3,3,3, 4,4,4,4,5,5, 5,5,0,0,0,0, 0,0,0,0,0,0])

|Segunda| SO, SO, SO, SO, MF, MF|
|Terça  | MF, MF, OC, OC, OC, OC|
|Quarta | CO, CO, CO, CO, IA, IA|
|Quinta | IA, IA, NA, NA, NA, NA|
|Sexta  | NA, NA, NA, NA, NA, NA|

Tempo Total de Espera Entre Aulas:  0
